<a href="https://colab.research.google.com/github/tanvi-ap/ClimateChange_StanceDetection/blob/main/tweet_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import requests
import io
import pandas as pd

import preprocessor as p

from datetime import datetime, date, timedelta

import numpy as np



In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
import pandas as pd
import glob
import os
import fnmatch

#df1 = pd.read_csv('/drive/MyDrive/covid_vaccine_tweets_extracted_20211202_043040.csv')
#df2 =

path = '/drive/MyDrive/'

for filename in os.listdir(path):
  #if fnmatch.fnmatch(filename,'covid_vaccine_tweets_extracted_*.csv'):
  if fnmatch.fnmatch(filename,'climate_change_tweets_extracted_*.csv'):
    #print(filename)
    df_from_file = pd.read_csv(path+'/'+filename,index_col=0) 
    final_df = pd.concat([df_from_file])

#type(df_from_file)

In [ ]:
final_df.head()

,username,location,text,hashtags,tweet_date
0,Peter Leighton-Jones Ⓥ 🏴󠁧󠁢󠁷󠁬󠁳󠁿🌱,"Leeds, England",@_Spinflight_ Global warming is just one sympt...,[],2021-11-26 23:45:50
1,ZNet,NaN,A root reality of injustice experience in the ...,[],2021-11-26 23:44:25
2,Playing For Change,The World,Join us December 5th on #PFCPlantATreeDay and ...,"[{'text': 'PFCPlantATreeDay', 'indices': [24, ...",2021-11-26 23:30:00
3,"Tom Gillispie, LET'S KEEP NATURE LOOKING NATURAL","Rural Hall, North Carolina, USA",#climate #change The 11TH HOUR FOR THE EARTH #...,"[{'text': 'climate', 'indices': [0, 8]}, {'tex...",2021-11-26 23:12:36
4,@drtimball #climate,Columnist-Landowners Magazine,Canada is the “worst performer” of G7 nations ...,[],2021-11-26 23:12:11


In [ ]:
#path = 'https://raw.githubusercontent.com/tanvi-ap/Covid19StanceDetection/main/covid19_vaccination.csv'
path = 'https://raw.githubusercontent.com/tanvi-ap/ClimateChange_StanceDetection/main/climatechange_tweets%20(2).csv'
r = requests.get(path)
tweets_df = pd.read_csv(io.StringIO(r.text),index_col=0)

tweets_df.head()

,username,location,text,hashtags,tweet_date
0,NOAAandNASAareRight,NaN,@wattsupwiththat Watts denies rising sea level...,[],2021-11-21 23:57:20
1,Neale Jones,Wellington,Challenged by Corin Dann about climate change ...,[],2021-11-21 23:56:34
2,Climate Signals,United States,A record high of 52% of respondents said they ...,[],2021-11-21 23:44:59
3,Gareth Smith,"Dunfermline, Fife KY11 8UH",@Tony__Heller Can you imagine living in a worl...,[],2021-11-21 23:36:27
4,Disequal,NaN,@larrythelefty @WhitePaleAle @arayeahs @VaushV...,[],2021-11-21 23:35:41


In [ ]:
final_tweets_df = pd.concat([tweets_df,final_df])

In [ ]:
final_tweets_df.shape

(24166, 5)

In [ ]:
final_tweets_df.head()

,username,location,text,hashtags,tweet_date
0,NOAAandNASAareRight,NaN,@wattsupwiththat Watts denies rising sea level...,[],2021-11-21 23:57:20
1,Neale Jones,Wellington,Challenged by Corin Dann about climate change ...,[],2021-11-21 23:56:34
2,Climate Signals,United States,A record high of 52% of respondents said they ...,[],2021-11-21 23:44:59
3,Gareth Smith,"Dunfermline, Fife KY11 8UH",@Tony__Heller Can you imagine living in a worl...,[],2021-11-21 23:36:27
4,Disequal,NaN,@larrythelefty @WhitePaleAle @arayeahs @VaushV...,[],2021-11-21 23:35:41


In [ ]:
final_tweets_df.sort_values(by='tweet_date').reset_index(drop = True)

,username,location,text,hashtags,tweet_date
0,K M Resist,USA,@dcminx The fact we call it climate change now...,[],2021-11-20 21:19:24
1,K M Resist,USA,@dcminx The fact we call it climate change now...,[],2021-11-20 21:19:24
2,K M Resist,USA,@dcminx The fact we call it climate change now...,[],2021-11-20 21:19:24
3,K M Resist,USA,@dcminx The fact we call it climate change now...,[],2021-11-20 21:19:24
4,K M Resist,USA,@dcminx The fact we call it climate change now...,[],2021-11-20 21:19:24
...,...,...,...,...,...
24161,Susan Brown,Western Australia,@Info_Aus @zalisteggall @ICAN_Vic You cannot d...,[],2021-12-03 23:09:27
24162,Daddio Donner 🎅💉🌊💙#BLM 🌈,NaN,@patriottakes Climate doesn't change when the ...,[],2021-12-03 23:37:31
24163,TODDO,"Mesa, AZ",@MsMeowMix @BFriedmanDC Do you think it will b...,[],2021-12-03 23:41:10
24164,Søren Lindgreen,Copenhagen,The practice today is referred to as “geoengin...,[],2021-12-03 23:44:46


In [ ]:
final_tweets_df['text_cleaned'] = final_tweets_df['text'].apply(lambda x: p.clean(x))

In [ ]:
final_tweets_df.head()

,username,location,text,hashtags,tweet_date,text_cleaned
0,NOAAandNASAareRight,NaN,@wattsupwiththat Watts denies rising sea level...,[],2021-11-21 23:57:20,Watts denies rising sea levels -- of course! T...
1,Neale Jones,Wellington,Challenged by Corin Dann about climate change ...,[],2021-11-21 23:56:34,Challenged by Corin Dann about climate change ...
2,Climate Signals,United States,A record high of 52% of respondents said they ...,[],2021-11-21 23:44:59,A record high of % of respondents said they ha...
3,Gareth Smith,"Dunfermline, Fife KY11 8UH",@Tony__Heller Can you imagine living in a worl...,[],2021-11-21 23:36:27,Can you imagine living in a world where climat...
4,Disequal,NaN,@larrythelefty @WhitePaleAle @arayeahs @VaushV...,[],2021-11-21 23:35:41,I absolutely believe climate change and global...


In [ ]:
tweets_df_final = final_tweets_df[(final_tweets_df['text'].str.contains("climate")) 
                            | (final_tweets_df['text'].str.contains("Climate"))]

len(tweets_df_final)

21133

In [ ]:
tweets_df = tweets_df_final.copy()

In [ ]:
# Clean tweet text with tweet-preprocessor
tweets_df['text_cleaned'] = tweets_df['text'].apply(lambda x: p.clean(x))

In [ ]:
# Remove duplicate tweets
tweets_df.drop_duplicates(subset='text_cleaned', keep="first", inplace = True)
len(tweets_df)

2243

In [ ]:
# Create timestamp for datetime of extraction
extract_datetime = datetime.today().strftime('%Y%m%d_%H%M%S')
filename = '/drive/MyDrive/final_covid_vaccine_tweets_extracted_' + extract_datetime + '.csv'

tweets_df_final.drop_duplicates(inplace = True)

# Export dataframe as csv file with above filename
tweets_df_final.to_csv(filename, index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# Remove unnecessary characters
# Note: Need to remove % as Stanford CoreNLP annotation encounters error if text contains some of these characters
punct =['%','/',':','\\','&amp;','&',';']

def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, '')
    return text

tweets_df['text_cleaned'] = tweets_df['text_cleaned'].apply(lambda x: remove_punctuations(x))

In [ ]:
# Drop tweets which have empty text field
tweets_df['text_cleaned'].replace('', np.nan, inplace=True)
tweets_df['text_cleaned'].replace(' ', np.nan, inplace=True)
tweets_df.dropna(subset=['text_cleaned'], inplace=True)
len(tweets_df)

2243

In [ ]:
tweets_df = tweets_df.reset_index(drop=True)
tweets_df.sample(5)

,username,location,text,hashtags,tweet_date,text_cleaned
990,Dave,"North Carolina, USA",@44MagnumBlue1 They're due for a name change a...,[],2021-11-25 00:27:30,They're due for a name change and rebranding o...
337,Alfie Arthur Archie,West Yorkshire,Global warming is fake and climate change is f...,[],2021-11-23 22:23:38,Global warming is fake and climate change is f...
1540,أبو الحسنات الصدیق,"London, Casablanca",Read this in-depth feature by @russianabout on...,"[{'text': 'climatechange', 'indices': [125, 13...",2021-11-29 15:32:51,Read this in-depth feature by on what lies ahe...
53,Trotsky like Wokester 🇨🇦,Lekwungen Land,@terry_truchan I don't need to know the scienc...,[],2021-11-21 18:03:30,I don't need to know the science of climate ch...
2059,Liberty Bell Society,The America's,"@blaw52 “Climate change”, formally known as “g...",[],2021-12-02 03:54:47,"Climate change, formally known as global warmi..."


In [ ]:
#tweets_df.to_csv('/drive/MyDrive/covid19_train.csv')
tweets_df.to_csv('/drive/MyDrive/climate_change_train.csv')